In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
tf.enable_eager_execution()

In [2]:
data=tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [55]:
text = open(data,'rb').read().decode(encoding ='utf-8')

In [56]:
print("Length of text {}".format(len(text)))

Length of text 1115394


In [57]:
print(text[:290])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citiz


In [58]:
vocab =sorted(set(text))

In [59]:
char2id = {u:i for i,u in enumerate(vocab)}
id2char = np.array(vocab)

In [60]:
text_as_int = np.array([char2id[l] for l in text])

In [61]:
text[0]

'F'

In [62]:
seq_length=100
example_per_epoch = len(text)

In [63]:
char_data = tf.data.Dataset.from_tensor_slices(text_as_int)

In [64]:
for i in char_data.take(4):
    print(id2char[i.numpy()])

F
i
r
s


In [65]:
sequences = char_data.batch(seq_length+1,drop_remainder=True)

In [66]:
for item in sequences.take(5):
  print(repr(''.join(id2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [67]:
def split_input_ex(inputs):
    input_data = inputs[:-1]
    target_data = inputs[1:]
    return input_data,target_data

dataset=sequences.map(split_input_ex)

In [68]:
for input_ex,target_ex in dataset.take(1):
    print("Input Ex {}".format(repr(''.join(id2char[input_ex]))))
    print("Target Ex {}".format(repr(''.join(id2char[target_ex]))))

Input Ex 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target Ex 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [69]:
steps_per_epoch = example_per_epoch
BATCH = 64
BUFFER=1000
dataset = dataset.shuffle(BUFFER).batch(BATCH, drop_remainder=True)

In [70]:
dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [71]:
import functools
rnn = functools.partial(tf.keras.layers.GRU,recurrent_activation='sigmoid')

In [74]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size,embedding_dim,batch_size,rnn_units):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size,embedding_dim,batch_input_shape=[batch_size,None]))
    model.add(rnn(rnn_units,return_sequences=True,recurrent_initializer='glorot_uniform',stateful=True))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

In [75]:
model = build_model(vocab_size,embedding_dim,BATCH,rnn_units)

In [77]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [78]:
for inputs,targets in dataset.take(1):
    example_batch_pred = model(inputs)
    print(example_batch_pred.shape)

(64, 100, 65)


In [89]:
sampled_indices=tf.random.categorical(example_batch_pred[0],num_samples=1)

In [91]:
sampled_indices = tf.squeeze(sampled_indices,axis=1)

In [92]:
sampled_indices

<tf.Tensor: id=16446, shape=(100,), dtype=int64, numpy=
array([12, 24, 11, 41,  3,  2, 53, 56, 44, 28, 61, 26, 48, 58, 51, 44, 47,
       60, 14,  3, 12, 47, 26, 42,  0, 15, 19, 40, 16, 40, 31, 41, 40, 64,
       31, 33, 46, 51, 40, 22, 13, 51, 48, 29, 19, 37, 21, 57, 36,  0, 18,
       49, 64,  8, 54, 21, 59, 44, 22, 37, 23, 24, 20, 64, 43, 31, 39,  7,
       39, 57, 43,  5, 23, 28, 18, 52, 33, 22, 38, 26, 32, 63, 58, 56,  4,
       14, 57, 62, 45, 42, 50, 40, 16, 61, 12, 55, 49, 59, 13, 10])>

In [95]:
print("Input {}".format(repr(''.join(id2char[inputs[0]]))))
print("Target {}".format(repr(''.join(id2char[targets[0]]))))
print("Output {}".format(repr(''.join(id2char[sampled_indices]))))

Input " a dangerous physic\nThat's sure of death without it, at once pluck out\nThe multitudinous tongue; let"
Target "a dangerous physic\nThat's sure of death without it, at once pluck out\nThe multitudinous tongue; let "
Output "?L;c$!orfPwNjtmfivB$?iNd\nCGbDbScbzSUhmbJAmjQGYIsX\nFkz.pIufJYKLHzeSa-ase'KPFnUJZNTytr&BsxgdlbDw?qkuA:"


In [96]:
def loss(labels,logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

ex_batch_loss = loss(targets,example_batch_pred)

In [98]:
ex_batch_loss.numpy().mean()

4.173992

In [100]:
model.compile(optimizer=tf.train.AdamOptimizer(),loss=loss)

In [101]:
checkpoint_dir = './training_checkpoints'
checkpoint_file = os.path.join(checkpoint_dir,'ckpt_{epoch}')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_file,save_weights_only=True)

In [ ]:
history = model.fit(dataset.repeat(),epochs=3,steps_per_epoch=steps_per_epoch,callbacks=[checkpoint])